In [ ]:
#%pylab qt
%pylab inline
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive

In [ ]:
cd MyDrive

In [ ]:
cd texture3-dev

In [ ]:
!python setup.py build
!python setup.py install

In [ ]:
from mpl_toolkits.mplot3d import axes3d
from TX import sym, euler

In [ ]:
def draw_unit(
    ax=None,
    m2v=np.identity(3),
    kws=dict(ls='-',c='orange',marker='None',alpha=0.5),linewidth=0.3):
  
    orig=np.array([0.5,0.5,0.5])

    o=np.dot(np.array([0,0,0]-orig),m2v)
    a=np.dot(np.array([1,0,0]-orig),m2v)
    b=np.dot(np.array([0,1,0]-orig),m2v)
    c=np.dot(np.array([0,0,1]-orig),m2v)
    d=np.dot(np.array([1,0,1]-orig),m2v)
    e=np.dot(np.array([0,1,1]-orig),m2v)
    f=np.dot(np.array([1,1,0]-orig),m2v)
    g=np.dot(np.array([1,1,1]-orig),m2v)

    if type(ax)==type(None):
        fig=plt.figure()
        ax1=fig.add_subplot(projection='3d')    
    else:
        ax1=ax

    dat=np.array([o,a]).T
    ax1.plot(*dat,**kws)
    dat=np.array([o,b]).T
    ax1.plot(*dat,**kws)
    dat=np.array([o,c]).T
    ax1.plot(*dat,**kws)
    dat=np.array([a,d]).T
    ax1.plot(*dat,**kws)
    dat=np.array([b,e]).T
    ax1.plot(*dat,**kws)
    dat=np.array([c,e]).T
    ax1.plot(*dat,**kws)
    dat=np.array([c,d]).T
    ax1.plot(*dat,**kws)
    dat=np.array([a,f]).T
    ax1.plot(*dat,**kws)
    dat=np.array([b,f]).T
    ax1.plot(*dat,**kws)
    dat=np.array([g,f]).T
    ax1.plot(*dat,**kws)
    dat=np.array([g,d]).T
    ax1.plot(*dat,**kws)
    dat=np.array([g,e]).T
    ax1.plot(*dat,**kws)

    ax1.set_xlabel('x')
    ax1.set_ylabel('y')
    ax1.set_zlabel('z')
    
    ax1.set_xticks([-1,0,1])
    ax1.set_yticks([-1,0,1])
    ax1.set_zticks([-1,0,1]) 

In [ ]:
def miller2vec(icrysym,isn,cvec,trans=np.identity(3)):
    if icrysym==2 or icrysym==3: ## hexagonal or trigonal
        isn[2]=isn[3]
        isb[0]=isb[0]-isb[2]
        isb[1]=isb[1]-isb[2]
        isb[2]=isb[3]
    sn=np.zeros(3)
    sn[2]= isn[2]/cvec[2,2]
    sn[0]=[isn[0]-cvec[2,0]*sn[2]]/cvec[0,0]
    sn[1]=[isn[1]-cvec[0,1]*sn[0]-cvec[2,1]*sn[2]]/cvec[1,1]
    sn_mag=sqrt((sn**2).sum())
    sn=sn/sn_mag
    sn=np.dot(sn,trans)
    return sn

def deco(axs):
    ax,ax1,ax2=axs[:]

    ## half-(north)sphere surface
    u, v = np.mgrid[0:2*np.pi:200j, 0:np.pi/2.:200j]
    x = np.cos(u)*np.sin(v)
    y = np.sin(u)*np.sin(v)
    z = np.cos(v)
    ax1.plot_surface(x, y, z, color="k",alpha=0.1)

    ## full sphere surface
    u, v = np.mgrid[0:2*np.pi:200j, 0:np.pi:200j]
    x = np.cos(u)*np.sin(v)
    y = np.sin(u)*np.sin(v)
    z = np.cos(v)
    ax.plot_surface(x, y, z, color="k",alpha=0.1)    
    
    ## for ax1, upper hemisphere
    ## for ax, full hemisphere

    th=np.linspace(-np.pi,np.pi,100)
    X=cos(th);Y=sin(th)
    ax1.plot(X,Y,'k-')
    ax.plot(X,Y,'k-')    

    ## x, y ticks
    for a in fig.axes:
      a.plot([0.95,1.05],[0,0],'m-')
      a.plot([0.0,0.0],[0.95,1.05],'b-')    

    ax2.plot(X,Y,'k-')
    ax2.set_axis_off()
    ax2.set_aspect('equal')

## Euler angles to transformation matrix

- Let's define the Euler angles (Z,X,Z)

In [ ]:
"""
        1: CUBIC                                                                                                                                                                                                                                                                               
        2: HEXAGONAL                                                                                                                                                                                                                                                                           
        3: TRIGONAL                                                                                                                                                                                                                                                                            
        4: TETRAGONAL                                                                                                                                                                                                                                                                          
        5: ORTHORHOMBIC                                                                                                                                                                                                                                                                        
        6: MONOCLINIC                                                                                                                                                                                                                                                                          
        7: TRICLINIC   """


In [ ]:
icrysym=1
isn=[1,1,0]
phi1=-20.
phi=15.
phi2=30.
cdim=[0.2,0.2,0.2]
cang=[90,90,90]

## figure canvas
fig=plt.figure(figsize=(12,3.4))
ax1=fig.add_subplot(131,projection='3d')  
ax2=fig.add_subplot(132,projection='3d')  
ax3=fig.add_subplot(133)
## draw crystal unit cell

## generate transformation matrix.
trans=euler.euler(phi1,phi,phi2,echo=False)
cvec=sym.cvec(cdim=cdim,cang=cang)
cvec_=np.dot(cvec,trans)
for ax in fig.axes[0:1]:
  draw_unit(ax,m2v=cvec_)

# crystal plane miller index
    
if icrysym==1:
    hs=sym.cubic()
else:
    hs=np.array([np.identity(3)])

mir=np.zeros((3,3))
for i in range(3):
  mir[i,i]=-1

## point mirrors
hs_=np.zeros((hs.shape[0]*2,3,3))
for i in range(hs.shape[0]):
  hs_[2*i,:,:]=hs[i,:,:]
  hs_[2*i+1,:,:]=matmul(hs[i,:,:],mir)
hs=hs_.copy()

south=np.array([0,0,-1])
sn_init=miller2vec(icrysym,isn,cvec)
for i in range(hs.shape[0]):
    h=hs[i,:,:]
    sn=np.dot(h,sn_init)
    sn=np.dot(sn,trans)
    o=np.array([0,0,0])
    d=np.array([o,sn]).T

    if sn[2]>=0:
        for ax in fig.axes[:2]:
          ax.plot([sn[0]],[sn[1]],[sn[2]],'o',c='r',mfc='None')
        ax1.plot(*d,c='lightgreen',alpha=1,lw=0.2,zorder=-100)          

        ## line connecting south pole to vector points projected on the sphere.        
        d=np.array([sn,south]).T
        ax2.plot(*d,'--',c='k',alpha=0.5,lw=0.2)

        ## stereographic projection
        X,Y=sn[0]/(1+sn[2]),sn[1]/(1+sn[2]) # northsphere to proejct
        kwd=dict(mfc='g',marker='o',mec='None')
        ax2.plot([X],[Y],[0],**kwd)
        
        ax3.plot(X,Y,**kwd)
        if i==0:ax3.text(X,Y,isn.__str__(),va='center',ha='left')
    else:
      ax1.plot(*d,c='lightgreen',alpha=1,lw=1)
      ax1.plot([sn[0]],[sn[1]],[sn[2]],'o',c='r',mfc='None')      

    
ax1.plot([south[0]],[south[1]],[south[2]],'k.')
ax2.plot([south[0]],[south[1]],[south[2]],'k.')
ax1.text(*south,'south pole',ha='left',va='top')
for ax in fig.axes[:2]:
  ax.text(1,0,0,'X-axis',c='m',ha='left',va='top')
  ax.text(0,1,0,'Y-axis',c='b',ha='left',va='bottom')
ax3.text(1,0,'X-axis',c='m',ha='left',va='center')
ax3.text(0,1,'Y-axis',c='b',ha='center',va='bottom')
ax1.set_axis_off()
ax2.set_axis_off()
## deco
deco(fig.axes)
fig.savefig('pf_stereoprojection.png',dpi=500,transparent=True)